In [1]:
#By Emmanuel Cocom
import pandas as pd
import numpy as np

### READ RAW DATA

In [2]:
#link to problem on kaggle
#https://www.kaggle.com/c/bike-sharing-demand/data

#get untouched dataset
bycle_df = pd.read_csv('train.csv')

#columns
bycle_df.columns

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count'],
      dtype='object')

In [3]:
#shape
bycle_df.shape

(10886, 12)

In [4]:
#df printed out to see raw data
bycle_df[:350:50]

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,1/1/11 0:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
50,1/3/11 5:00,1,0,1,1,6.56,6.820,47,19.0012,0,3,3
100,1/5/11 9:00,1,0,1,1,9.02,9.850,37,22.0028,6,109,115
150,1/7/11 13:00,1,0,1,2,8.20,9.090,37,19.0012,9,64,73
200,1/9/11 15:00,1,0,0,1,9.02,9.850,35,23.9994,5,77,82
250,1/11/11 19:00,1,0,1,3,6.56,11.365,93,0.0000,0,51,51
300,1/13/11 23:00,1,0,1,1,4.92,6.820,50,12.9980,1,14,15


### MUST DROP BOTH CASUAL AND REGISTERED COLUMNS.  LEAKAGE VARIABLES 

They are another representation of the label we are trying to predict 'count' in two columns

registered + casual = count (label)

we will not know registered, casual data in ever in any real life testing because they are the label 'count' we are trying predict! 

Keeping them jeapordizes the integrity and usefulness of our model as it will rely on those columns to predict the label, but they are the label

In [5]:
#Drop casual
bycle_df.drop('casual', axis=1, inplace=True)

#DROP registered
bycle_df.drop('registered', axis=1, inplace=True)
              
              
bycle_df.columns
#both casual and registered are gone

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'count'],
      dtype='object')

## Unsupported Formats:    Datetime column cannot be used in it's current format:  date hour:00.

#### Data will be extracted from datetime column to form three new columns: 

hour : 0-23

day_of_year: 0 - 365 (but raw data only provides up to 364)

weekday: 0-6 (0-Sun, 6-Sat)

In [6]:
from datetime import datetime

#extract hour from datetime
bycle_df['hour'] = bycle_df.datetime.apply(lambda x : x.split()[1].split(':')[0])

#extract day_of_year from datetime
bycle_df['day_of_year'] = bycle_df.datetime.apply(lambda x : datetime.strptime(x,  '%m/%d/%y %H:00').strftime('%-j'))

#extract weekday from datetime
bycle_df['weekday']= bycle_df.datetime.apply(lambda x : datetime.strptime(x,  '%m/%d/%y %H:00').strftime('%w'))

#bycle_df.datetime.apply(lambda x : datetime.strptime(x,  '%m/%d/%y %H:00').strftime('%w'))

#extract weekday from datetime
bycle_df['year']= bycle_df.datetime.apply(lambda x : datetime.strptime(x,  '%m/%d/%y %H:00').strftime('%Y')).to_frame('year')

#extract weekday from datetime
bycle_df['month']= bycle_df.datetime.apply(lambda x : datetime.strptime(x,  '%m/%d/%y %H:00').strftime('%-m')).to_frame('month')


bycle_df.head()

#adding month year features to see if model improves if we can predict by month or year 
#year_df = bycle_df.datetime.apply(lambda x : datetime.strptime(x,  '%m/%d/%y %H:00').strftime('%Y')).to_frame('year')
#datetime_df = bycle_df.datetime.apply(lambda x: datetime.strptime(x,  '%m/%d/%y %H:00').strftime('%-m')).to_frame('month')
#month_year_df = pd.concat([year_df ,datetime_df ], axis=1)
#month_year_df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,hour,day_of_year,weekday,year,month
0,1/1/11 0:00,1,0,0,1,9.84,14.395,81,0.0,16,0,1,6,2011,1
1,1/1/11 1:00,1,0,0,1,9.02,13.635,80,0.0,40,1,1,6,2011,1
2,1/1/11 2:00,1,0,0,1,9.02,13.635,80,0.0,32,2,1,6,2011,1
3,1/1/11 3:00,1,0,0,1,9.84,14.395,75,0.0,13,3,1,6,2011,1
4,1/1/11 4:00,1,0,0,1,9.84,14.395,75,0.0,1,4,1,6,2011,1


### MISSING VALUES and Corrupt data values

In [7]:
#HELPER FUNCTIONS, TO TEST FOR MISSING OR CORRUPT DATA IN EACH COLUMN/FEATURE



#helper for string data
def check_if_missing_strings(value):
    if value == None or value.strip() == '' or value.lower() == 'nan' or value == np.nan:
        print(value, j, ' is the missing value')
        return np.nan
    else:
        return value
    

    
#Helper for numerical data
def check_if_empty_ints(value):
    if value == '' or value == None or value == 'nan' or value == np.nan:
        print('missing value')
    else: 
        pass
    
def check_if_less_one(value):
    if value <1:
        print('less than one, check it out!')
        
def check_if_greater_than(value, limit):
    if  value > limit:
        print('less than one, check it out!')
def check_if_less_than(value, limit):
    if value < limit:
        print('less than one, check it out!')
def check_if_less_than_x_greater_than_y(value, x_limit, y_limit):
    if value < x_limit:
        print(value)
        print(' is dangerous data and less than ', x_limit)
    if value > y_limit:
        print(value)
        print(' is dangerous data and greater than ', y_limit)

#checking for both missing and corrupt data( data that does not make sense )
#datetime stamp :: clear
bycle_df.datetime.apply(lambda x: check_if_missing_strings(x)) #results show NO missing values

#season :: clear
bycle_df.season.apply(lambda x: check_if_empty_ints(x))#results show NO missing values
bycle_df.season.apply(lambda x: check_if_less_one(x))#results show NO corrupt values

#holiday :: clear
bycle_df.holiday.apply(lambda x: check_if_empty_ints(x))#results show NO missing values
bycle_df.holiday.apply(lambda x: check_if_less_than_x_greater_than_y(x, 0, 1))#results show NO corrupt values
bycle_df.holiday.unique()

#workingday:: clear
bycle_df.workingday.apply(lambda x: check_if_empty_ints(x))#results show NO missing values
bycle_df.workingday.apply(lambda x: check_if_less_than_x_greater_than_y(x, 0, 1))#results show NO corrupt values
bycle_df.workingday.unique()

#weather:: clear
bycle_df.weather.apply(lambda x: check_if_empty_ints(x))#results show NO missing values
bycle_df.weather.apply(lambda x: check_if_less_than_x_greater_than_y(x, 1, 4))#results show NO corrupt values
bycle_df.weather.unique()

#atemp :: clear
bycle_df.atemp.apply(lambda x: check_if_empty_ints(x))#results show NO missing values
bycle_df.atemp.apply(lambda x: check_if_less_than(x, 0.1))#results show NO corrupt values
bycle_df.atemp.unique()

#-humidity :: NOT CLEAR! --> NO MISSING DATA: HOWEVER ---> CORRUPT DATA: YES 
bycle_df.humidity.apply(lambda x: check_if_empty_ints(x)) #results show NO missing values
bycle_df.humidity.unique() #results show YES Corrupt Data, as it's impossible to have 0 humidity on earth

#windspeed :: clear
bycle_df.windspeed.apply(lambda x: check_if_empty_ints(x)) #results show NO missing values
bycle_df.windspeed.unique() #results show No Missing Data, as it is possible to have 0 windspeed

#hour :: clear
bycle_df.hour.apply(lambda x: check_if_empty_ints(x)) #results show NO missing values
bycle_df.hour.apply(lambda x: check_if_less_than_x_greater_than_y(int(x), 0, 23))#results show NO corrupt data values
bycle_df.hour.unique() #results show No Missing Data,

#day_of_year :: clear
bycle_df.day_of_year.apply(lambda x: check_if_empty_ints(x)) #results show NO missing values
bycle_df.day_of_year.apply(lambda x: check_if_less_than_x_greater_than_y(int(x), int(0), int(365)))#results show NO corrupt data values
bycle_df.day_of_year.unique() #results show No Missing Data,

#Weekday :: clear
bycle_df.weekday.apply(lambda x: check_if_empty_ints(x)) #results show NO missing values
bycle_df.weekday.apply(lambda x: check_if_less_than_x_greater_than_y(int(x), int(0), int(6)))#results show NO corrupt data values
bycle_df.weekday.unique() #results show No Missing Data,
print('testing done')


testing done


In [8]:
print(bycle_df.shape)
bycle_df.head()

(10886, 15)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,hour,day_of_year,weekday,year,month
0,1/1/11 0:00,1,0,0,1,9.84,14.395,81,0.0,16,0,1,6,2011,1
1,1/1/11 1:00,1,0,0,1,9.02,13.635,80,0.0,40,1,1,6,2011,1
2,1/1/11 2:00,1,0,0,1,9.02,13.635,80,0.0,32,2,1,6,2011,1
3,1/1/11 3:00,1,0,0,1,9.84,14.395,75,0.0,13,3,1,6,2011,1
4,1/1/11 4:00,1,0,0,1,9.84,14.395,75,0.0,1,4,1,6,2011,1


### Fixing Corrupt Data - By replacing it with column average

In [9]:
#fixing humidity by adding average of the column to all zero values (as zero is impossible value it means the data is just missing)
import numpy as np
def nan_if_zero(value):
    if value == 0:
        return np.nan
    return value


def mean_if_nan(value, mean):
    if value == 0:
        return mean
    else:
        return value
    
    
#verify corrupt data
bycle_df['humidity'].unique() 

#changes zero corrupt data to numpy.nan value
random_var = bycle_df.humidity.apply(lambda x: nan_if_zero(x))

#finds mean of column, ignores nan values by default
mean_humid = random_var.mean() 


#apply mean to all zero corrupt data values in original df column humidity
bycle_df['humidity'] = bycle_df.humidity.apply(lambda x: mean_if_nan(int(x), int(mean_humid)))

#DATA IS CLEARED NOW --> NO CORRUPTION IN HUMIDITY COLUMN
bycle_df['humidity'].unique()



array([ 81,  80,  75,  86,  76,  77,  72,  82,  88,  87,  94, 100,  71,
        66,  57,  46,  42,  39,  44,  47,  50,  43,  40,  35,  30,  32,
        64,  69,  55,  59,  63,  68,  74,  51,  56,  52,  49,  48,  37,
        33,  28,  38,  36,  93,  29,  53,  34,  54,  41,  45,  92,  62,
        58,  61,  60,  65,  70,  27,  25,  26,  31,  73,  21,  24,  23,
        22,  19,  15,  67,  10,   8,  12,  14,  13,  17,  16,  18,  20,
        85,  83,  84,  78,  79,  89,  97,  90,  96,  91])

In [10]:
#Numerical features - scaled

### One Hot Encode categorical features that are non-binary

### Looking at all possible values for all columns that need one Hot Encoding

In [11]:
#categorical features
cat_features = ['season', 'holiday','workingday', 'weather', 'hour', 'month']


#looking for non-binary categorical features to OneHotEncode
non_binary_cat_features = {}
for x in cat_features:
    if len(bycle_df[x].unique()) > 2: #only if they have 3 or more possible values
        non_binary_cat_features[x] = bycle_df[x].unique()

print('The following need to go through One Hot Encoded Transformation:\n')
for x,y in non_binary_cat_features.items():
    print(x, 'has the features ', y)
    
#features that do no not need to be one hot encoded
non_encoded_features = ['holiday','workingday', 'temp','atemp', 'humidity', 'windspeed', 'day_of_year', 'weekday', 'year']



The following need to go through One Hot Encoded Transformation:

season has the features  [1 2 3 4]
weather has the features  [1 2 3 4]
hour has the features  ['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15'
 '16' '17' '18' '19' '20' '21' '22' '23']
month has the features  ['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12']


#### Column 'Year needs to be changed to binary form'

In [12]:
def year_to_bin(value):
    if value == '2011':
        return 0
    return 1

bycle_df['year']= bycle_df.year.apply(lambda x : year_to_bin(x))

In [13]:
bycle_df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,hour,day_of_year,weekday,year,month
0,1/1/11 0:00,1,0,0,1,9.84,14.395,81,0.0,16,0,1,6,0,1
1,1/1/11 1:00,1,0,0,1,9.02,13.635,80,0.0,40,1,1,6,0,1
2,1/1/11 2:00,1,0,0,1,9.02,13.635,80,0.0,32,2,1,6,0,1
3,1/1/11 3:00,1,0,0,1,9.84,14.395,75,0.0,13,3,1,6,0,1
4,1/1/11 4:00,1,0,0,1,9.84,14.395,75,0.0,1,4,1,6,0,1


### Column 'hour' will be put into 4 bins of 6 hours. This is reduce the number of features that will be created due to OneHotEncoding. Features created will be 4 instead of 24.

In [14]:
#HELPER FUNCTION TO PUT HOUR VALUES INTO BINS
def four_hour_bins(hour):
    hour = int(hour)
    if hour <=5:
        return 1
    elif hour <=11:
        return 2
    elif hour <=17:
        return 3
    else:
        return 4

bycle_df['hour'] = bycle_df.hour.apply(lambda x : four_hour_bins(x))
print('The possible choicse of column hour after putting into bins')
print(bycle_df['hour'].unique())

The possible choicse of column hour after putting into bins
[1 2 3 4]


In [15]:
bycle_df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,count,hour,day_of_year,weekday,year,month
0,1/1/11 0:00,1,0,0,1,9.84,14.395,81,0.0,16,1,1,6,0,1
1,1/1/11 1:00,1,0,0,1,9.02,13.635,80,0.0,40,1,1,6,0,1
2,1/1/11 2:00,1,0,0,1,9.02,13.635,80,0.0,32,1,1,6,0,1
3,1/1/11 3:00,1,0,0,1,9.84,14.395,75,0.0,13,1,1,6,0,1
4,1/1/11 4:00,1,0,0,1,9.84,14.395,75,0.0,1,1,1,6,0,1


#### Actual OHE Proces

In [16]:
#One hot encoding

from sklearn.preprocessing import OneHotEncoder
#create OneHotEncoder object for each one
one_hot_encod_season = OneHotEncoder()
one_hot_encod_weather = OneHotEncoder()
one_hot_encod_hour = OneHotEncoder()
one_hot_encod_month = OneHotEncoder()



#transform values to OneHotEncoding values with new columns for each feature
x_season = one_hot_encod_season.fit_transform(bycle_df.season.values.reshape(-1,1)).toarray()
x_weather = one_hot_encod_weather.fit_transform(bycle_df.weather.values.reshape(-1,1)).toarray()
x_hour = one_hot_encod_weather.fit_transform(bycle_df.weather.values.reshape(-1,1)).toarray()
x_month = one_hot_encod_month.fit_transform(bycle_df.month.values.reshape(-1,1)).toarray()

print(x_hour)

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


In [17]:
#Make a data data frame for each categorical feature that was one hot encoded using numpy results and proper column names
df_bycle_ohe_season = pd.DataFrame(x_season, columns = [' spring,', 'summer', 'fall', 'winter'])
df_bycle_ohe_weather = pd.DataFrame(x_weather, columns = [' clear', 'mist', 'light', 'heavy_rain'])
df_bycle_ohe_hour = pd.DataFrame(x_hour, columns = [' EarlyMorning', 'Morning', 'Evening', 'Night'])
df_bycle_ohe_month = pd.DataFrame(x_month, columns = ['Jan', 'Feb', 'March', 'April', 'May', 'June', 'July', 'August', 'Sept','Oct','Nov','Dec'])



print(df_bycle_ohe_season[:300:50])
print('\n\n\n')
print(df_bycle_ohe_weather[:300:50])

print('\n\n\n')
print(df_bycle_ohe_hour[:300:50])

print('\n\n\n')
print(df_bycle_ohe_month[:300:50])

#concatenate all the individual one hot encoded dataframes into one dataframe
df_bycle_ohe_feature_matrix = pd.concat([df_bycle_ohe_season,df_bycle_ohe_weather ], axis=1)
df_bycle_ohe_feature_matrix = pd.concat([df_bycle_ohe_feature_matrix, df_bycle_ohe_hour ], axis=1)
df_bycle_ohe_feature_matrix = pd.concat([df_bycle_ohe_feature_matrix, df_bycle_ohe_month ], axis=1)



#print out new df containing onehotencoding columns and values
print('\n\n\nOneHotEncoded DF:\n\n')
print(df_bycle_ohe_feature_matrix[:300:50])
non_encoded_feature_matrix = bycle_df[non_encoded_features]

      spring,  summer  fall  winter
0         1.0     0.0   0.0     0.0
50        1.0     0.0   0.0     0.0
100       1.0     0.0   0.0     0.0
150       1.0     0.0   0.0     0.0
200       1.0     0.0   0.0     0.0
250       1.0     0.0   0.0     0.0




      clear  mist  light  heavy_rain
0       1.0   0.0    0.0         0.0
50      1.0   0.0    0.0         0.0
100     1.0   0.0    0.0         0.0
150     0.0   1.0    0.0         0.0
200     1.0   0.0    0.0         0.0
250     0.0   0.0    1.0         0.0




      EarlyMorning  Morning  Evening  Night
0              1.0      0.0      0.0    0.0
50             1.0      0.0      0.0    0.0
100            1.0      0.0      0.0    0.0
150            0.0      1.0      0.0    0.0
200            1.0      0.0      0.0    0.0
250            0.0      0.0      1.0    0.0




     Jan  Feb  March  April  May  June  July  August  Sept  Oct  Nov  Dec
0    1.0  0.0    0.0    0.0  0.0   0.0   0.0     0.0   0.0  0.0  0.0  0.0
50   1.0  0.0    0.0 

### Features & Labels

#### New Feature Matrix-- Combining numerical and OneHotEncoded Featuress

In [18]:
#Labels
label = bycle_df['count']
print(label.shape)
print('label is count: \n')
print(label[0:300:50])

#combine non encoded and encoded feature matrices
bycle_feature_matrix = pd.concat([non_encoded_feature_matrix, df_bycle_ohe_feature_matrix], axis = 1)


#columns of new feature matrix
print('\n\nfeature matrix columns')
print('column names are \n', bycle_feature_matrix.columns)

#print(bycle_feature_matrix.head())
#df_to_be_used_later = bycle_feature_matrix.copy()
#df_to_be_used_later = pd.concat([df_to_be_used_later ,df_bycle_ohe_weather ], axis=1)
#df_to_be_used_later.head()

(10886,)
label is count: 

0       16
50       3
100    115
150     73
200     82
250     51
Name: count, dtype: int64


feature matrix columns
column names are 
 Index(['holiday', 'workingday', 'temp', 'atemp', 'humidity', 'windspeed',
       'day_of_year', 'weekday', 'year', ' spring,', 'summer', 'fall',
       'winter', ' clear', 'mist', 'light', 'heavy_rain', ' EarlyMorning',
       'Morning', 'Evening', 'Night', 'Jan', 'Feb', 'March', 'April', 'May',
       'June', 'July', 'August', 'Sept', 'Oct', 'Nov', 'Dec'],
      dtype='object')


#### We will make a Models Based Of Each Month instead of a model for every month in the year, so a df split will created for each month

In [19]:
#first bring it back together, so labels are paired off correctly with f matrix as data is filtered and split
df_with_label = pd.concat([bycle_feature_matrix, label], axis = 1)
df_with_label.head()

,holiday,workingday,temp,atemp,humidity,windspeed,day_of_year,weekday,year,"spring,",...,April,May,June,July,August,Sept,Oct,Nov,Dec,count
0,0,0,9.84,14.395,81,0.0,1,6,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16
1,0,0,9.02,13.635,80,0.0,1,6,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40
2,0,0,9.02,13.635,80,0.0,1,6,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32
3,0,0,9.84,14.395,75,0.0,1,6,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13
4,0,0,9.84,14.395,75,0.0,1,6,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [20]:
df_splits = []
months = ['Jan', 'Feb', 'March', 'April', 'May', 'June', 'July', 'August', 'Sept', 'Oct', 'Nov', 'Dec']

#0- feature_matrix 1-label
for month in months:
    df_month_fmatrix = df_with_label[df_with_label[month] == 1.0]#filter out rows for only that month

    label_month = df_month_fmatrix['count'] #labels are extracted for filtered rows
    
    del df_month_fmatrix['count'] #label is dropped from feature matrix
    df_splits.append([df_month_fmatrix, label_month]) #feature matrix and labels are put into a list
print('\n\n\n\n')

for x in range(len(df_splits)):
    print('Month: ', months[x])
    print('shape fmatrix', df_splits[x][0].shape)
    print('shape label is', df_splits[x][1].shape)
    print('\n')
    







Month:  Jan
shape fmatrix (884, 33)
shape label is (884,)


Month:  Feb
shape fmatrix (901, 33)
shape label is (901,)


Month:  March
shape fmatrix (901, 33)
shape label is (901,)


Month:  April
shape fmatrix (909, 33)
shape label is (909,)


Month:  May
shape fmatrix (912, 33)
shape label is (912,)


Month:  June
shape fmatrix (912, 33)
shape label is (912,)


Month:  July
shape fmatrix (912, 33)
shape label is (912,)


Month:  August
shape fmatrix (912, 33)
shape label is (912,)


Month:  Sept
shape fmatrix (909, 33)
shape label is (909,)


Month:  Oct
shape fmatrix (911, 33)
shape label is (911,)


Month:  Nov
shape fmatrix (911, 33)
shape label is (911,)


Month:  Dec
shape fmatrix (912, 33)
shape label is (912,)




### NORMALIZING DATA

#### Normalize data for each month

In [21]:
from sklearn import preprocessing

#normalize data
for x in range(len(df_splits)):
    #scale it -> d type changes to numpy array
    scaled_feature_matrix_month_numpyarray = preprocessing.scale(df_splits[x][0])

    #change back to df
    df_month_scaled = pd.DataFrame(scaled_feature_matrix_month_numpyarray, columns = df_splits[x][0].columns)
    
    #store back the scaled data back into list.
    df_splits[x][0] = df_month_scaled

print('sample of list of stored monthly dataframes and label\n\n')
print(df_splits[0][0].head())
print(df_splits[0][1][:5:])

sample of list of stored monthly dataframes and label


    holiday  workingday          temp     atemp  humidity  windspeed  \
0 -0.295518   -1.283241 -4.389351e-16  0.479102  1.436178    -1.7049   
1 -0.295518   -1.283241 -2.026208e-01  0.321980  1.378933    -1.7049   
2 -0.295518   -1.283241 -2.026208e-01  0.321980  1.378933    -1.7049   
3 -0.295518   -1.283241 -4.389351e-16  0.479102  1.092707    -1.7049   
4 -0.295518   -1.283241 -4.389351e-16  0.479102  1.092707    -1.7049   

   day_of_year   weekday      year   spring, ...   March  April  May  June  \
0    -1.640371  1.600362 -1.025204       0.0 ...     0.0    0.0  0.0   0.0   
1    -1.640371  1.600362 -1.025204       0.0 ...     0.0    0.0  0.0   0.0   
2    -1.640371  1.600362 -1.025204       0.0 ...     0.0    0.0  0.0   0.0   
3    -1.640371  1.600362 -1.025204       0.0 ...     0.0    0.0  0.0   0.0   
4    -1.640371  1.600362 -1.025204       0.0 ...     0.0    0.0  0.0   0.0   

   July  August  Sept  Oct  Nov  Dec  
0  

## ALGORITHM 1: RANDOM FOREST

In [22]:
bycle_feature_matrix.head()

,holiday,workingday,temp,atemp,humidity,windspeed,day_of_year,weekday,year,"spring,",...,March,April,May,June,July,August,Sept,Oct,Nov,Dec
0,0,0,9.84,14.395,81,0.0,1,6,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,9.02,13.635,80,0.0,1,6,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,9.02,13.635,80,0.0,1,6,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,9.84,14.395,75,0.0,1,6,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,9.84,14.395,75,0.0,1,6,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
print(bycle_feature_matrix.columns)

Index(['holiday', 'workingday', 'temp', 'atemp', 'humidity', 'windspeed',
       'day_of_year', 'weekday', 'year', ' spring,', 'summer', 'fall',
       'winter', ' clear', 'mist', 'light', 'heavy_rain', ' EarlyMorning',
       'Morning', 'Evening', 'Night', 'Jan', 'Feb', 'March', 'April', 'May',
       'June', 'July', 'August', 'Sept', 'Oct', 'Nov', 'Dec'],
      dtype='object')


### X_train, X_test, y_train, y_test for each month of year - going to be used from here on out, for any individual runs

In [26]:
from sklearn.model_selection import train_test_split
monthly_train_test_splits = []
for month in df_splits:
    X_train, X_test, y_train, y_test = train_test_split(month[0], month[1], test_size=0.25, 
                                                    random_state=4)
    monthly_train_test_splits.append([X_train, X_test, y_train, y_test])


In [27]:
print('sample of monthly split for first month')
print('X_train shape is', monthly_train_test_splits[0][0].shape)
print('y_train shape is',monthly_train_test_splits[0][2].shape )

print('X_test shape is', monthly_train_test_splits[0][1].shape)
print('y_test shape is', monthly_train_test_splits[0][3].shape)

sample of monthly split for first month
X_train shape is (663, 33)
y_train shape is (663,)
X_test shape is (221, 33)
y_test shape is (221,)


### INDIVIDUAL RUN

In [28]:
from sklearn.ensemble import RandomForestRegressor

In [235]:
from sklearn import metrics
import numpy as np

#kaggle requested metric

def rmsle(y, y_):
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y_]))
    calc = (log1 - log2) ** 2
    return np.sqrt(np.mean(calc))


#rmse, feature importance, predictions below

monthly_forests_rmse = {}
monthly_f_importance = {}
monthly_predictions = {}
monthly_forests_rmsle = {}

for x in range(len(monthly_train_test_splits)):
    rf = RandomForestRegressor(n_estimators = 200, random_state = 1)
    rf.fit(monthly_train_test_splits[x][0], monthly_train_test_splits[x][2]);
    predictions = rf.predict(monthly_train_test_splits[x][1])
    mse = metrics.mean_squared_error(monthly_train_test_splits[x][3], predictions)
    rmse = np.sqrt(mse)
    monthly_forests_rmse[months[x]] = rmse
    
    #save feature importance for each month
    monthly_f_importance[months[x]]  = pd.Series(rf.feature_importances_,index=bycle_feature_matrix.columns).sort_values(ascending=False)
    monthly_predictions[months[x]] = predictions
    monthly_forests_rmsle[months[x]] = rmsle(monthly_train_test_splits[x][3], predictions)
    #save predictions


#### Feature Importance

In [236]:
#print('\n\n\n')
print('The following are the feature importance generated by random forest model for each month\n\n')
for x,y in monthly_f_importance.items():
    print('month: ', x, '\n\nfeature_importance:\n\n', y)
    print('\n\n')


The following are the feature importance generated by random forest model for each month


month:  Jan 

feature_importance:

 humidity         0.206621
atemp            0.158797
temp             0.150113
windspeed        0.145716
year             0.131038
day_of_year      0.094667
weekday          0.051582
workingday       0.012860
 EarlyMorning    0.011955
 clear           0.010667
mist             0.008404
Morning          0.007747
light            0.003218
holiday          0.003203
Evening          0.002950
heavy_rain       0.000279
Night            0.000181
Nov              0.000000
April            0.000000
Oct              0.000000
Sept             0.000000
August           0.000000
July             0.000000
June             0.000000
May              0.000000
March            0.000000
 spring,         0.000000
summer           0.000000
fall             0.000000
winter           0.000000
Feb              0.000000
Jan              0.000000
Dec              0.000000
dtype: float64


### Evaluating our results

#### Using kaggle's requested metric for evaluation

In [237]:
def rmsle(y, y_):
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y_]))
    calc = (log1 - log2) ** 2
    return np.sqrt(np.mean(calc))

In [238]:
for x,y in monthly_forests_rmsle.items():
    print('month: ', x, 'rmsle: ', y)

month:  Jan rmsle:  1.1441221545732299
month:  Feb rmsle:  1.2485996959766934
month:  March rmsle:  1.2735875583661602
month:  April rmsle:  1.2069250277325583
month:  May rmsle:  1.173552112915778
month:  June rmsle:  1.0849185983704237
month:  July rmsle:  1.0764016194646775
month:  August rmsle:  0.9600500628371289
month:  Sept rmsle:  1.1902804446522923
month:  Oct rmsle:  1.0315300447771811
month:  Nov rmsle:  0.9962746079233606
month:  Dec rmsle:  1.1412956090136617


#### Using RMSE Metric for evaulation

In [239]:
for x,y in monthly_forests.items():
    print('month: ', x, ' rmse: ', y)


month:  Jan  rmse:  74.89381715263706
month:  Feb  rmse:  85.4993037846434
month:  March  rmse:  120.08627336853009
month:  April  rmse:  138.14658566477954
month:  May  rmse:  132.49636160232748
month:  June  rmse:  162.2874058273357
month:  July  rmse:  135.25299135203394
month:  August  rmse:  152.53471828595428
month:  Sept  rmse:  154.41697217981778
month:  Oct  rmse:  172.46187130213815
month:  Nov  rmse:  119.65030188176351
month:  Dec  rmse:  129.3119442955345


### CROSS VALIDATION RUN

In [240]:
from sklearn.cross_validation import cross_val_score

crossv_montly_forests_rmse_list = {}
crossv_montly_forests_rmse = {}

for month in range(len(df_splits)):
    rf_cv = RandomForestRegressor(random_state = 42)
    mse_list= cross_val_score(rf_cv, df_splits[month][0], df_splits[month][1], cv=10, scoring='neg_mean_squared_error')
    mse_list_positive = -mse_list
    rmse_list = np.sqrt(mse_list_positive)
    rmse_mean = rmse_list.mean()
    
    #save monthly rmse list
    crossv_montly_forests_rmse_list[months[month]] = rmse_list 
    
    #save monthly rmse mean
    crossv_montly_forests_rmse[months[month]] = rmse_mean
    
for x,y in crossv_montly_forests_rmse_list.items():
      print('\nmonth: ', x, '\nrmse_list: ', y)

for x,y in crossv_montly_forests_rmse.items():
      print('month: ', x, 'rmse: ', y)


month:  Jan 
rmse_list:  [ 66.05386176  52.69369237  50.81627491  42.69705624  52.12410047
 130.90965576 135.47988806 118.66919988 102.10105246 116.60739807]

month:  Feb 
rmse_list:  [ 58.195902    74.24852421  59.66965483  74.49663167 105.2944364
 173.83289527 125.49480429 110.93043343 143.49157857 145.55917621]

month:  March 
rmse_list:  [121.45833989  68.57220798  71.33211317  70.60169725 157.77578824
 143.24107035 161.38762184 161.58263578 204.63334012 241.5570799 ]

month:  April 
rmse_list:  [133.78884762 108.79620539 106.92618787  97.60263314 120.65221775
 202.83194093 191.55387878 204.27527971 189.97977579 232.42701344]

month:  May 
rmse_list:  [178.34441328 130.25340598 147.28723133 126.79137956 156.11256732
 209.14935577 205.73578385 191.74532898 174.98969214 192.63911736]

month:  June 
rmse_list:  [185.89345113 144.43673683 152.30227817 173.01578797 175.74791525
 175.24750077 252.71228016 223.8563479  206.81838851 212.98362435]

month:  July 
rmse_list:  [120.05155138 1

### IMPROVING ACCURACY ATTEMPT - FEATURE REDUCTION -FAILED TO IMPROVE ACCURACY

#### Manual Feature Reduction... Checking RMSE With up to 7 best features.

#FEATURE IMPORTANCE- Best features below
    
#best 5 features for each month model
####     ['humidity','atemp','temp','windspeed','year' ] #1 - Jan
####     ['humidity','temp','windspeed','atemp','day_of_year']#Feb
####     ['temp','atemp','humidity','windspeed','year']#3 March
####    ['humidity','windspeed','temp','atemp','day_of_year']#April
####     ['atempt','humidity','windspeed','day_of_year','temp'],#May
####     ['humidity','windspeed','year', 'day_of_year', 'temp',],#6 June
####     ['temp','humidity','day_of_year','windspeed','atemp'],#July
####     ['temp','humidity','windspeed','day_of_year','year'],#August
####     ['humidity','atemp','windspeed','temp','day_of_year'],#9 Sept
####     ['humidity','windspeed','atemp','temp','day_of_year'],#10 - Oct
####     ['humidity','temp','windspeed','day_of_year','temp',],#11 - #Nov
####     ['humidity','temp','windspeed','day_of_year','atemp',],#12 - December

#### cross validation and feature reduction

In [241]:
#best 5 features for each month model
best_features = [
    ['humidity','atemp','temp','windspeed','year' ],#1 - Jan
    ['humidity','temp','windspeed','atemp','day_of_year'],#Feb
    ['temp','atemp','humidity','windspeed','year'],#3 March
    ['humidity','windspeed','temp','atemp','day_of_year'],#April
    ['atempt','humidity','windspeed','day_of_year','temp'],#May
    ['humidity','windspeed','year', 'day_of_year', 'temp',],#6 June
    ['temp','humidity','day_of_year','windspeed','atemp'],#July
    ['temp','humidity','windspeed','day_of_year','year'],#August
    ['humidity','atemp','windspeed','temp','day_of_year'],#9 Sept
    ['humidity','windspeed','atemp','temp','day_of_year'],#10 - Oct
    ['humidity','temp','windspeed','day_of_year','temp',],#11 - Nov
    ['humidity','temp','windspeed','day_of_year','atemp',],#12 - December
]


In [242]:
from sklearn.cross_validation import cross_val_score

crossv_montly_forests_feature_reduction_rmse_list = {}
crossv_montly_forests_feature_reduction_rmse = {}

for month in range(len(df_splits)):
    rf_cv = RandomForestRegressor(random_state = 42)
    #line below filters out all other columsn other than 5 best for each model of each year
    mse_list= cross_val_score(rf_cv, df_splits[month][0][best_features[0]], df_splits[month][1], cv=10, scoring='neg_mean_squared_error')
    mse_list_positive = -mse_list
    rmse_list = np.sqrt(mse_list_positive)
    rmse_mean = rmse_list.mean()
    
    #save monthly rmse list
    crossv_montly_forests_feature_reduction_rmse_list[months[month]] = rmse_list 
    
    #save monthly rmse mean
    crossv_montly_forests_feature_reduction_rmse[months[month]] = rmse_mean
    
#for x,y in crossv_montly_forests_feature_reduction_rmse_list.items():
#      print('\nmonth: ', x, '\nrmse_list: ', y)

for x,y in crossv_montly_forests_feature_reduction_rmse.items():
      print('month: ', x, 'rmse: ', y)

month:  Jan rmse:  91.55157200867895
month:  Feb rmse:  107.21732300847218
month:  March rmse:  137.84884730684948
month:  April rmse:  154.53206524505202
month:  May rmse:  179.03714858542975
month:  June rmse:  188.59647411941782
month:  July rmse:  178.5739272058808
month:  August rmse:  172.86927030438568
month:  Sept rmse:  182.94046735810554
month:  Oct rmse:  176.55853860447914
month:  Nov rmse:  159.00466128233376
month:  Dec rmse:  166.64236498872023


### IMPROVING ACCURACY ATTEMPT -ADA BOOST - SUCCESS

In [243]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

In [244]:

ada_boost_montly_rmse = {}
for x in range(len(monthly_train_test_splits)):
    #rf_ab= RandomForestRegressor(n_estimators = 100, random_state = 3)
    ada_boost_reg = AdaBoostRegressor(RandomForestRegressor(n_estimators = 100, random_state = 3), n_estimators=100, random_state=3)
    #rf_ab.fit(X_train, y_train)
    ada_boost_reg.fit(monthly_train_test_splits[x][0], monthly_train_test_splits[x][2])
    
    #rf_predictions = rf_ab.predict(X_test)
    abr_predictions = ada_boost_reg.predict(monthly_train_test_splits[x][1])
    
    #rf_rmse= sqrt(mean_squared_error(y_test, rf_predictions))
    # abr_predictions_rmse =  sqrt(mean_squared_error(monthly_train_test_splits[x][3], abr_predictions))
    
    mse = metrics.mean_squared_error(monthly_train_test_splits[x][3], abr_predictions)
    rmse = np.sqrt(mse)
    ada_boost_montly_rmse[months[x]] = rmse
    
    
for x,y in ada_boost_montly_rmse.items():
    print('\nmonth: \n', x , ' rmse: ', y, '\n')


month: 
 Jan  rmse:  72.75198565955834 


month: 
 Feb  rmse:  85.00368184696093 


month: 
 March  rmse:  123.83674114007582 


month: 
 April  rmse:  152.41919988140077 


month: 
 May  rmse:  136.6884980660333 


month: 
 June  rmse:  163.89954391915765 


month: 
 July  rmse:  140.75404431400784 


month: 
 August  rmse:  154.5471237820825 


month: 
 Sept  rmse:  160.4746116351526 


month: 
 Oct  rmse:  173.26387256837526 


month: 
 Nov  rmse:  121.22522651585474 


month: 
 Dec  rmse:  125.74212010272952 



In [245]:
from statistics import mean
mean(ada_boost_montly_rmse.values())
print('averaging the rmse of all 12 models we get', mean(ada_boost_montly_rmse.values()))

averaging the rmse of all 12 models we get 134.2172207859491


### IMPROVING ACCURACY ATTEMPT -ADA BOOST CROSS VALIDATION- SUCCESS 

In [246]:
from sklearn.cross_validation import cross_val_score

crossv_adab_montly_forests_rmse_list = {}
crossv_adab_montly_forests_rmse = {}

for month in range(len(df_splits)):
    ada_boost = AdaBoostRegressor(RandomForestRegressor(n_estimators = 100, random_state = 3), n_estimators = 100, random_state = 6)
    mse_ada = cross_val_score(ada_boost, df_splits[month][0], df_splits[month][1], cv=10, scoring='neg_mean_squared_error')

    mse_ada_positive = - mse_ada
    rmse_ada_list = np.sqrt(mse_ada_positive)
    #print(rmse_ada)
    rmse_cv_ada= rmse_ada_list.mean()
    #print(accuracy_cv_ada)


    #save monthly rmse list
    crossv_adab_montly_forests_rmse_list[months[month]] = rmse_ada_list
    
    #save monthly rmse mean
    crossv_adab_montly_forests_rmse[months[month]] = rmse_cv_ada
    
#for x,y in crossv_adab_montly_forests_rmse_list.items():
#      print('month: ', x, '\nrmsle_list: ', y)

for x,y in crossv_adab_montly_forests_rmse.items():
      print('month: ', x, 'rmsle: ', y)

month:  Jan rmsle:  80.34241622447799
month:  Feb rmsle:  101.6952145952317
month:  March rmsle:  131.79033989998982
month:  April rmsle:  146.25477739337475
month:  May rmsle:  164.73312683434733
month:  June rmsle:  175.93547545797875
month:  July rmsle:  161.1246920955337
month:  August rmsle:  167.97093012502083
month:  Sept rmsle:  174.07783215694514
month:  Oct rmsle:  169.6338942835485
month:  Nov rmsle:  145.05192582303482
month:  Dec rmsle:  155.16918106530642


In [247]:
mean(crossv_adab_montly_forests_rmse.values())

147.81498382956582

### IMPROVING ACCURACY ATTEMPT - PCA - DIMENSIONALITY REDUCTION - SUCCESS

In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from math import sqrt
import numpy as np
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error

pca_montly_rmse = {}
for x in range(len(monthly_train_test_splits)):
    
    #grabbing data from split done early on
    X_train = monthly_train_test_splits[x][0]
    y_train = monthly_train_test_splits[x][2]
    
    X_test = monthly_train_test_splits[x][1]
    y_test = monthly_train_test_splits[x][3]
    
    
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = pd.DataFrame(scaler.transform(X_train), index=X_train.index.values, columns=X_train.columns.values)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test), index=X_test.index.values, columns=X_test.columns.values)
    
    
    
    pca = PCA() #create pca object
    pca.fit(X_train)
    
    cpts = pd.DataFrame(pca.transform(X_train))
    
    #print(cpts) #still same amount of columns as training
    x_axis = np.arange(1, pca.n_components_+1)
    #print(x_axis) # still same amount of columns as training
    
    pca_scaled = PCA()
    pca_scaled.fit(X_train_scaled)
    #print(pca_scaled)
    cpts_scaled = pd.DataFrame(pca.transform(X_train_scaled))

    
    #from sklearn.ensemble import RandomForestRegressor
    
    rf = RandomForestRegressor(n_estimators=500, oob_score=True, random_state=6)
    #print(X_train.head())
    #rf.fit(X_train, y_train)
    rf.fit(cpts_scaled, y_train)
    
    predicted_train = rf.predict(X_train)
    predicted_test = rf.predict(X_test)

    rf_rmse= sqrt(mean_squared_error(y_test, predicted_test))
    
    pca_montly_rmse[months[x]] = rf_rmse



In [32]:

for x,y in pca_montly_rmse.items():
    print('month is: ', x, ' rmse is ', y)

month is:  Jan  rmse is  74.38190886132988
month is:  Feb  rmse is  85.13935702765245
month is:  March  rmse is  119.90178528318302
month is:  April  rmse is  137.60061971098386
month is:  May  rmse is  132.40643456731658
month is:  June  rmse is  161.02288902867068
month is:  July  rmse is  134.58636278004485
month is:  August  rmse is  152.79645956212025
month is:  Sept  rmse is  154.46558177289586
month is:  Oct  rmse is  173.068933488444
month is:  Nov  rmse is  119.33586609971165
month is:  Dec  rmse is  127.9653620508366


#### ADA PCA RF

In [58]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from math import sqrt
import numpy as np
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import mean_squared_error

ab_pca_montly_rmse = {}    
for x in range(len(monthly_train_test_splits)):
    #grabbing data from split done early on
    X_train = monthly_train_test_splits[x][0]
    y_train = monthly_train_test_splits[x][2]
    
    X_test = monthly_train_test_splits[x][1]
    y_test = monthly_train_test_splits[x][3]
    
    
    #create the scaler and fit it using training data
    scaler = StandardScaler().fit(X_train)
    
    #create df with trained scaled data 
    X_train_scaled = pd.DataFrame(scaler.transform(X_train), index=X_train.index.values, columns=X_train.columns.values)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test), index=X_test.index.values, columns=X_test.columns.values)
    
    
    pca_rf = PCA() #create pca object
    pca_rf.fit(X_train)#pass x_train data
    
    cpts = pd.DataFrame(pca_rf.transform(X_train))
    
    #print(cpts) #still same amount of columns as training
    x_axis = np.arange(1, pca_rf.n_components_+1)
    #print(x_axis) # still same amount of columns as training
    
    pca_scaled = PCA()
    pca_scaled.fit(X_train_scaled)
    #print(pca_scaled)
    cpts_scaled = pd.DataFrame(pca.transform(X_train_scaled))
    
    
    
    
    rf = RandomForestRegressor(n_estimators=100, oob_score=True, random_state=6)
    rf.fit(X_train, y_train)
    predicted_train = rf.predict(X_train)
    predicted_test = rf.predict(X_test)
    
    
    
    ada_boost_reg2 = AdaBoostRegressor(rf, n_estimators=100, random_state=3)
    ada_boost_reg2.fit(X_train, y_train)
    abr_predictions = ada_boost_reg2.predict(X_test)
    ab_pca_rmse= sqrt(mean_squared_error(y_test, abr_predictions))
    #print('x',x, 'rmse is', ab_pca_rmse)
    ab_pca_montly_rmse[months[x]] = ab_pca_rmse

print('\n\n')
for x,y in ab_pca_montly_rmse.items():
    print('month is: ', x, ' rmse is ', y)




month is:  Jan  rmse is  72.75198565955834
month is:  Feb  rmse is  85.00368184696093
month is:  March  rmse is  123.83674114007582
month is:  April  rmse is  152.41919988140077
month is:  May  rmse is  136.6884980660333
month is:  June  rmse is  163.89954391915765
month is:  July  rmse is  140.75404431400784
month is:  August  rmse is  154.5471237820825
month is:  Sept  rmse is  160.4746116351526
month is:  Oct  rmse is  173.26387256837526
month is:  Nov  rmse is  121.22522651585474
month is:  Dec  rmse is  125.74212010272952
